In [128]:
from langchain.utilities.duckduckgo_search import DuckDuckGoSearchAPIWrapper
import yfinance
import json


def get_ticker(inputs):
    ddg = DuckDuckGoSearchAPIWrapper()
    company_name = inputs["company_name"]
    return ddg.run(f"Ticker symbol of {company_name}")


def get_income_statement(inputs):
    ticker = inputs["ticker"]
    stock = yfinance.Ticker(ticker)
    return json.dumps(stock.income_stmt.to_json())


def get_balance_sheet(inputs):
    ticker = inputs["ticker"]
    stock = yfinance.Ticker(ticker)
    return json.dumps(stock.income_stmt.to_json())


def get_daily_stock_performance(inputs):
    ticker = inputs["ticker"]
    stock = yfinance.Ticker(ticker)
    return json.dumps(stock.history(period="3mo").to_json())


functoins_map = {
    "get_ticker": get_ticker,
    "get_income_statement": get_income_statement,
    "get_balance_sheet": get_balance_sheet,
    "get_daily_stock_performance": get_daily_stock_performance,
}

functions = [
    {
        "type": "function",
        "function": {
            "name": "get_ticker",
            "description": "Given the name of a company returns its ticker symbol",
            "parameters": {
                "type": "object",
                "properties": {
                    "company_name": {
                        "type": "string",
                        "description": "The name of the company",
                    }
                },
                "required": ["company_name"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_income_statement",
            "description": "Given a ticker symbol (i.e AAPL) returns the company's income statement.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "Ticker symbol of the company",
                    },
                },
                "required": ["ticker"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_balance_sheet",
            "description": "Given a ticker symbol (i.e AAPL) returns the company's balance sheet.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "Ticker symbol of the company",
                    },
                },
                "required": ["ticker"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_daily_stock_performance",
            "description": "Given a ticker symbol (i.e AAPL) returns the performance of the stock for the last 100 days.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "Ticker symbol of the company",
                    },
                },
                "required": ["ticker"],
            },
        },
    },
]

get_income_statement({"ticker": "AAPL"})

'"{\\"1727654400000\\":{\\"Tax Effect Of Unusual Items\\":0.0,\\"Tax Rate For Calcs\\":0.240912,\\"Normalized EBITDA\\":134661000000.0,\\"Net Income From Continuing Operation Net Minority Interest\\":93736000000.0,\\"Reconciled Depreciation\\":11445000000.0,\\"Reconciled Cost Of Revenue\\":210352000000.0,\\"EBITDA\\":134661000000.0,\\"EBIT\\":123216000000.0,\\"Net Interest Income\\":null,\\"Interest Expense\\":null,\\"Interest Income\\":null,\\"Normalized Income\\":93736000000.0,\\"Net Income From Continuing And Discontinued Operation\\":93736000000.0,\\"Total Expenses\\":267819000000.0,\\"Total Operating Income As Reported\\":123216000000.0,\\"Diluted Average Shares\\":null,\\"Basic Average Shares\\":null,\\"Diluted EPS\\":null,\\"Basic EPS\\":null,\\"Diluted NI Availto Com Stockholders\\":93736000000.0,\\"Net Income Common Stockholders\\":93736000000.0,\\"Net Income\\":93736000000.0,\\"Net Income Including Noncontrolling Interests\\":93736000000.0,\\"Net Income Continuous Operations\

In [130]:
import openai as client

# assistant = client.beta.assistants.create(
#     name="Investor Assistant",
#     instructions="You help user do research on publicly traded companies and you help them decide if they should buy the stock or not.",
#     model="gpt-3.5-turbo-0125",
#     tools = functions
# )

assistant_id = "asst_W2n1XTFiMov4kmyAk1i7rmKM"

In [206]:
thread = client.beta.threads.create()

thread_id = thread.id

message = client.beta.threads.messages.create(
    thread_id=thread_id,
    role= "user",
    content="I want to know if Health Catalyst stock is a good buy."
)

In [207]:
run = client.beta.threads.runs.create(
    thread_id=thread_id,
    assistant_id=assistant_id,
)

run_id = run.id

In [205]:
import json

def get_run(run_id, thread_id):
    return client.beta.threads.runs.retrieve(
        run_id=run_id,
        thread_id=thread_id
    )

def get_messages(thread_id):
    messages = client.beta.threads.messages.list(thread_id=thread_id)
    messages = list(messages)
    messages.reverse()
    for message in messages:
        print(f"{message.role}:{message.content[0].text.value}")


def get_tool_output(run_id, thread_id):
    run = get_run(run_id, thread_id)
    outputs = []
    for action in run.required_action.submit_tool_outputs.tool_calls:
        action_id = action.id
        function = action.function
        print(f"Calling function {function.name} with arg {function.arguments}")
        outputs.append(
            {
                "output": functoins_map[function.name](json.loads(function.arguments)),
                "tool_call_id": action_id,
            }
        )
    return outputs 

def submit_tool_output(run_id, thread_id):
    outputs = get_tool_output(run_id, thread_id)
    return client.beta.threads.runs.submit_tool_outputs(
        run_id=run_id,
        thread_id=thread_id,
        tool_outputs=outputs
    )

def send_message(thread_id, content):
    return client.beta.threads.messages.create(
        thread_id=thread_id,
        role="user",
        content=content,
    )

In [ ]:
# from typing_extensions import override
# from openai import AssistantEventHandler

# # First, we create a EventHandler class to define
# # how we want to handle the events in the response stream.


# class EventHandler(AssistantEventHandler):

#     def on_text_created(self, text) -> None:
#         print(f"\nassistant > ", end="", flush=True)

#     def on_text_delta(self, delta, snapshot):
#         print(delta.value, end="", flush=True)

#     def on_tool_call_created(self, tool_call):
#         print(f"\nassistant > {tool_call.type}\n", flush=True)

#     def on_tool_call_delta(self, delta, snapshot):
#         if delta.type == "code_interpreter":
#             if delta.code_interpreter.input:
#                 print(delta.code_interpreter.input, end="", flush=True)
#                 if delta.code_interpreter.outputs:
#                     print(f"\n\noutput >", flush=True)
#                     for output in delta.code_interpreter.outputs:
#                         if output.type == "logs":
#                             print(f"\n{output.logs}", flush=True)


# # Then, we use the `stream` SDK helper
# # with the `EventHandler` class to create the Run
# # and stream the response.

# with client.beta.threads.runs.stream(
#     thread_id=thread.id,
#     assistant_id=assistant_id,
#     instructions="You help user do research on publicly traded companies and you help them decide if they should buy the stock or not.",
#     event_handler=EventHandler(),
# ) as stream:
#     stream.until_done()

# stream.current_run_step_snapshot


assistant > function



In [208]:
get_messages(thread_id)

user:I want to know if Health Catalyst stock is a good buy.


In [209]:
get_run(run_id, thread_id).status

'requires_action'

In [ ]:
submit_tool_output(run_id, thread_id)

Calling function get_ticker with arg {"company_name":"Health Catalyst"}
[{'output': "Health Catalyst has received a consensus rating of Moderate Buy. The company's average rating score is 2.75, and is based on 9 buy ratings, 3 hold ratings, and no sell ratings. Amount of Analyst Coverage. Health Catalyst has been the subject of 6 research reports in the past 90 days, demonstrating strong analyst interest in this stock. See the latest Health Catalyst Inc stock price (HCAT:XNAS), related news, valuation, dividends and more to help you make your investing decisions. Stock analysis for Health Catalyst Inc (HCAT:NASDAQ GS) including stock price, stock chart, company news, key statistics, fundamentals and company profile. Health Catalyst, Inc. provides data and analytics technology and services to healthcare organizations. Its offerings include data and analytics platform, a commercial-grade data and analytics platform for the healthcare sector; AI and data science, providing integration of 

'requires_action'